In [47]:
def Strategy_3():
    import tushare as ts
    import os
    import datetime
    import pymongo
    import pandas

    client = pymongo.MongoClient("127.0.0.1", 27017)
    
    stocks = client.Common.Stock.find({})
    today = datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
    found = []
    minDays = 1000

    for item in stocks:
        code = item["code"]
        collection = client.Trade[code]
        res = list(collection.find({}, limit=10, sort=[("date", pymongo.DESCENDING)]))
        if len(res) < 10:
            continue
        days = (today - res[-1]["date"]).days
        if days > minDays:
            continue
        if res[0]["close"] < res[1]["close"]:
            continue
        if days < minDays:
            minDays = days
            found = []
        v10 = 0
        volumeDays = 2
        vv = 0
        p5 = 0
        for i in range(10):
            v10 = v10 + res[i]["volume"]
            if i < volumeDays:
                vv = vv + res[i]["volume"]
            if i < 5:
                p5 = p5 + res[i]["close"]
        v10 = int(v10 / 10)
        vv = int(vv / volumeDays)
        p5 = int(p5 / 5)
        close = res[0]["close"]
        if vv >= v10 * 2 and close >= p5:
            item = {
                "code": code,
                "name": item["name"],
                "volume_" + str(volumeDays): vv,
                "volume_10": v10,
                "price_today": close,
                "price_5": p5,
                "factor": (1.0 * vv / v10),
            }
            found.append(item)

        found = sorted(found, key=lambda item: item["factor"], reverse=True)

    if len(found) > 0:
        print pandas.DataFrame(found[:10])
    else:
        print("No Result")

In [48]:
Strategy_3()

     code    factor  name  price_5  price_today  volume_10  volume_2
0  300634  3.794619  彩讯股份       47        54.00      58389    221564
1  002220  3.449339  天宝食品        6         7.38      21318     73533
2  002077  3.133448  大港股份       10        11.77      36059    112989
3  002290  2.988721  中科新材       11        11.77      19416     58029
4  002436  2.907861  兴森科技        5         5.83     271437    789301
5  600468  2.901667  百利电气        5         6.29      47278    137185
6  002931  2.882117  锋龙股份       22        41.98       1247      3594
7  002631  2.810230  德尔未来        8         9.69      65279    183449
8  600562  2.799158  国睿科技       24        27.69      69393    194242
9  002806  2.755910  华锋股份       21        25.63      36548    100723
